In [1]:
from sklearn.pipeline import Pipeline
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.svm import SVC

In [3]:
df = pd.read_csv('../Cases/Wisconsin/BreastCancer.csv', index_col=0)
df

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,Benign
63375,9,1,2,6,4,10,7,7,2,Malignant
76389,10,4,7,2,2,8,6,1,1,Malignant
95719,6,10,10,10,8,10,7,10,7,Malignant
128059,1,1,1,1,2,5,5,1,1,Benign
...,...,...,...,...,...,...,...,...,...,...
1369821,10,10,10,10,5,10,10,10,7,Malignant
1371026,5,10,10,10,4,10,5,6,3,Malignant
1371920,5,1,1,1,2,1,3,2,1,Benign


In [4]:
x,y = df.drop('Class', axis = 1), df['Class']

In [5]:
x_train, x_test, y_train,y_test = train_test_split(x,y,random_state=25, test_size=0.3, stratify=y)

Without scaling

In [6]:
svm = SVC(kernel='linear')
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9714285714285714


With Standard Scaling

In [7]:
scaler = StandardScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9666666666666667


With MinMax Scaling

In [8]:
scaler = MinMaxScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9714285714285714


using all three scales at one

In [9]:
scales = [StandardScaler(), MinMaxScaler(), None]

for scaler in scales:
    pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    print(f'{scaler}: {accuracy_score(y_test, y_pred)}')

StandardScaler(): 0.9666666666666667
MinMaxScaler(): 0.9714285714285714
None: 0.9714285714285714


# on hr dataset

In [10]:
hr = pd.read_csv('../datasets/HR_comma_sep.csv')
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [21]:
x,y = hr.drop('left', axis = 1), hr['left']
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state = 25, test_size=0.3,stratify=y)

Using pipeline and changing the kernal

In [24]:
ohe = OneHotEncoder(sparse_output=False, drop = 'first').set_output(transform='pandas')
col_trnf = ColumnTransformer(
    [('OHE', ohe, make_column_selector(dtype_include=object))],
    remainder='passthrough',
    verbose_feature_names_out=False
)
scaler = MinMaxScaler()

#using linear as kernal
svm = SVC(kernel='linear')
pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

#using poly as kernal
svm = SVC(kernel='poly')
pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))


#using sigmoid as kernal
svm = SVC(kernel='sigmoid')
pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.7821738164036453
0.9370971326961547
0.6797066014669927


Variations with c value

In [29]:
c_value = np.linspace(0.01, 5, 15)
scores = []
pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])
for c in tqdm(c_value):
    svm = SVC(kernel = 'linear', C=c)
    pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    scores.append([c, accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['C Value', 'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)

100%|██████████| 15/15 [00:53<00:00,  3.59s/it]


,C Value,accuracy Score
0,0.010000,0.762169
14,5.000000,0.781952
13,4.643571,0.781952
12,4.287143,0.781952
11,3.930714,0.781952
10,3.574286,0.781952
9,3.217857,0.781952
8,2.861429,0.781952
3,1.079286,0.782174
5,1.792143,0.782396


Applying different kernels

In [32]:
kernels = ['rbf', 'linear', 'poly', 'sigmoid']
scores = []
for k in tqdm(kernels):
    svm = SVC(kernel=k)
    pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    scores.append([k, accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['Kernel', 'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)


100%|██████████| 4/4 [00:12<00:00,  3.03s/it]


,Kernel,accuracy Score
0,rbf,0.943321
2,poly,0.937097
1,linear,0.782174
3,sigmoid,0.679707


changing c values

In [36]:
#using degree,  c values and kernals as well
degree = [2,3,4]
c_value = np.linspace(0.01, 5, 10)
kernels = ['rbf', 'linear', 'poly', 'sigmoid']
scores = []
# for k in tqdm(kernels):
for d in tqdm(degree):
    for c in c_value:
        svm = SVC( C=c, degree=d)
        pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

        pipe.fit(x_train, y_train)
        y_pred = pipe.predict(x_test)
        scores.append(['poly',c,d, accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['Kernel','C Value','degree' ,'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)

100%|██████████| 3/3 [01:28<00:00, 29.36s/it]


,Kernel,C Value,degree,accuracy Score
9,poly,5.000000,2,0.947766
6,poly,3.336667,2,0.947766
29,poly,5.000000,4,0.947766
26,poly,3.336667,4,0.947766
19,poly,5.000000,3,0.947766
16,poly,3.336667,3,0.947766
8,poly,4.445556,2,0.947544
18,poly,4.445556,3,0.947544
28,poly,4.445556,4,0.947544
25,poly,2.782222,4,0.946877


changing gamma values

In [ ]:
#using degree,  c values and kernals as well
gamma = np.linspace(0.01, 5, 10)
c_value = np.linspace(0.01, 5, 10)
scores = []
# for k in tqdm(kernels):
for g in tqdm(gamma):
    for c in c_value:
        svm = SVC( kernel = 'rbf', C=c, gamma=d)
        pipe = Pipeline([('TRNF', col_trnf), ('SCL', scaler), ("SVM", svm)])

        pipe.fit(x_train, y_train)
        y_pred = pipe.predict(x_test)
        scores.append(['rbf',c,d, accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['Kernel','C Value','degree' ,'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)

100%|██████████| 10/10 [05:20<00:00, 32.09s/it]


,Kernel,C Value,degree,accuracy Score
19,poly,5.00,4,0.968660
29,poly,5.00,4,0.968660
9,poly,5.00,4,0.968660
39,poly,5.00,4,0.968660
69,poly,5.00,4,0.968660
...,...,...,...,...
50,poly,0.01,4,0.781952
60,poly,0.01,4,0.781952
70,poly,0.01,4,0.781952
80,poly,0.01,4,0.781952
